<a href="https://colab.research.google.com/github/Davichobacter/frist-repo/blob/main/OCR_DoblePagina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update -qq
!apt-get install -y tesseract-ocr tesseract-ocr-spa tesseract-ocr-eng poppler-utils
!pip install -q pytesseract pdf2image pillow opencv-python pypdf

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
tesseract-ocr-eng is already the newest version (1:4.00~git30-7274cfa-1.1).
tesseract-ocr-eng set to manually installed.
The following NEW packages will be installed:
  poppler-utils tesseract-ocr-spa
0 upgraded, 2 newly installed, 0 to remove and 82 not upgraded.
Need to get 1,137 kB of archives.
After this operation, 3,006 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.12 [186 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-spa all 1:4.00~git30-7274cfa-1.1 [951 kB]
Fetched 1,137 kB in 1s (1,243 kB/s)
Selecting previou

In [2]:
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import cv2
import numpy as np
from pypdf import PdfWriter, PdfReader
import os

PDF_IN = "/content/El-Capital-Vol-5-Libro-II-Karl-Marx-Cap-20.pdf"
PDF_OUT = "Cap-20-El-Capital-Vol-5-Libro-II.pdf"
LANG = "spa+eng"
DPI = 350  # 300-400 recomendado para libros escaneados
SPLIT_RATIO = 0.5  # 0.5 = mitad exacta; ajusta si el pliegue no está centrado
MARGIN_PX = 20     # recorta un poquito alrededor del pliegue para evitar texto partido

# Si el layout es variado, psm 3; si es texto normal tipo libro, psm 4 o 6 suelen ir bien
TESS_CONFIG = "--oem 1 --psm 4"

def preprocess(pil_img: Image.Image) -> Image.Image:
    """Preprocesado moderado para OCR en escaneos de libro."""
    img = np.array(pil_img)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Reduce ruido y mejora contraste
    gray = cv2.medianBlur(gray, 3)

    # Binarización (Otsu)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    return Image.fromarray(th)

def split_book_page(pil_img: Image.Image, split_ratio=0.5, margin_px=20):
    """Divide la imagen en izquierda y derecha, recortando un margen del centro."""
    w, h = pil_img.size
    split_x = int(w * split_ratio)

    left_box  = (0, 0, max(split_x - margin_px, 0), h)
    right_box = (min(split_x + margin_px, w), 0, w, h)

    left = pil_img.crop(left_box)
    right = pil_img.crop(right_box)
    return left, right

pages = convert_from_path(PDF_IN, dpi=DPI)

writer = PdfWriter()
temp_files = []

out_page_count = 0

for i, page in enumerate(pages, start=1):
    # Divide en 2 páginas de libro
    left, right = split_book_page(page, SPLIT_RATIO, MARGIN_PX)

    for side_name, side_img in [("L", left), ("R", right)]:
        clean = preprocess(side_img)

        pdf_bytes = pytesseract.image_to_pdf_or_hocr(
            clean,
            lang=LANG,
            config=TESS_CONFIG,
            extension="pdf"
        )

        tmp_name = f"_tmp_pdfpage_{i}_{side_name}.pdf"
        with open(tmp_name, "wb") as f:
            f.write(pdf_bytes)

        reader = PdfReader(tmp_name)
        for p in reader.pages:
            writer.add_page(p)
            out_page_count += 1

        temp_files.append(tmp_name)

    print(f"OCR página PDF {i}/{len(pages)} -> +2 páginas (izq/der) listo")

with open(PDF_OUT, "wb") as f:
    writer.write(f)

# Limpieza
for f in temp_files:
    try:
        os.remove(f)
    except OSError:
        pass

print(f"✅ PDF buscable generado: {PDF_OUT}  (páginas salida: {out_page_count})")

OCR página PDF 1/59 -> +2 páginas (izq/der) listo
OCR página PDF 2/59 -> +2 páginas (izq/der) listo
OCR página PDF 3/59 -> +2 páginas (izq/der) listo
OCR página PDF 4/59 -> +2 páginas (izq/der) listo
OCR página PDF 5/59 -> +2 páginas (izq/der) listo
OCR página PDF 6/59 -> +2 páginas (izq/der) listo
OCR página PDF 7/59 -> +2 páginas (izq/der) listo
OCR página PDF 8/59 -> +2 páginas (izq/der) listo
OCR página PDF 9/59 -> +2 páginas (izq/der) listo
OCR página PDF 10/59 -> +2 páginas (izq/der) listo
OCR página PDF 11/59 -> +2 páginas (izq/der) listo
OCR página PDF 12/59 -> +2 páginas (izq/der) listo
OCR página PDF 13/59 -> +2 páginas (izq/der) listo
OCR página PDF 14/59 -> +2 páginas (izq/der) listo
OCR página PDF 15/59 -> +2 páginas (izq/der) listo
OCR página PDF 16/59 -> +2 páginas (izq/der) listo
OCR página PDF 17/59 -> +2 páginas (izq/der) listo
OCR página PDF 18/59 -> +2 páginas (izq/der) listo
OCR página PDF 19/59 -> +2 páginas (izq/der) listo
OCR página PDF 20/59 -> +2 páginas (izq/